In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import progressbar

### I) Import des deux jeux de données et première visualisation.

In [2]:
raw_datas = pd.read_csv("./data wavestone/FD_MOBPRO_2018.csv", sep=";")
raw_datas_2 = pd.read_excel('./data wavestone/wave_densite.xlsx')

/Users/victor/opt/anaconda3/envs/wavestone/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


On jette un premier coup d'oeil sur la manière dont sont organisées les données. 

In [3]:
data_shape = raw_datas.shape 
print("Dimension du tableau de données : ", data_shape)
#data_cols = raw_datas.columns 
raw_datas.head()

Dimension du tableau de données :  (7932895, 32)


,COMMUNE,ARM,DCFLT,DCLT,AGEREVQ,CS1,DEROU,DIPL,EMPL,ILT,...,REGION,REGLT,SEXE,STAT,STOCD,TP,TRANS,TYPL,TYPMR,VOIT
0,1001,ZZZZZ,99999,1001,20,6,Z,16,16,1,...,84,84,1,10,10,2,1,1,41,1
1,1001,ZZZZZ,99999,1001,60,1,Z,13,21,1,...,84,84,1,21,10,1,1,1,11,1
2,1001,ZZZZZ,99999,1001,50,4,Z,14,16,1,...,84,84,2,10,10,2,1,1,41,1
3,1001,ZZZZZ,99999,1001,50,2,Z,13,22,1,...,84,84,1,22,10,1,1,1,41,1
4,1001,ZZZZZ,99999,1001,55,4,Z,13,16,1,...,84,84,2,10,10,2,5,1,12,1


In [4]:
data_shape_2 = raw_datas_2.shape 
print("Dimension du tableau de données : ", data_shape_2)
#data_cols_2 = raw_datas_2.columns
#print(data_cols_2)
raw_datas_2.head()

Dimension du tableau de données :  (34965, 6)


,\nCode \nCommune\n,Libellé des communes\n,Degré de \nDensité de la commune\n,Région\n,Population \nmunicipale \n2018,Unnamed: 5
0,1001,L'Abergement-Clémenciat,3,84,771,NaN
1,1002,L'Abergement-de-Varey,4,84,253,NaN
2,1004,Ambérieu-en-Bugey,2,84,14204,NaN
3,1005,Ambérieux-en-Dombes,3,84,1720,NaN
4,1006,Ambléon,4,84,112,NaN


### II) On vérifie que nos deux jeux de données sont homogènes. 

On récupère la liste des identifiants des communes dans chacun des deux jeux de données. On compare ces deux listes pour voir si elles diffèrent. Pour cela, on s'assure que tous les éléments de la liste sont bien du meme type.

In [5]:
#On normalise les types (on met tous les identifiants en chaines de caractères) :  
raw_datas = raw_datas.astype({"COMMUNE": str})
raw_datas_2 = raw_datas_2.astype({"\nCode \nCommune\n": str})

#On récupère les identifiants de ville dans deux listes :
communes_ids = raw_datas["COMMUNE"].unique()
communes_ids_2 = raw_datas_2["\nCode \nCommune\n"].unique()

#les longueurs des listes diffèrent : 
print("Longueur de la liste 1 : ",len(communes_ids), 
      "\nLongueur de la liste 2 : ",len(communes_ids_2),
      "\n\n")   

#On capture les éléments de la première liste qui ne sont pas dans la deuxième. 
#Et réciproquement ceux qui sont dans la deuxième mais pas la première. 
in_one_not_in_two = []
in_two_not_in_one = []
for idd in communes_ids:
    if idd not in communes_ids_2:
        in_one_not_in_two.append(idd)
for idd in communes_ids_2:
    if idd not in communes_ids:
        in_two_not_in_one.append(idd)
        
print("Élements de la liste 1 non présents dans la liste 2 : ", in_one_not_in_two, 
      " \nÉlements de la liste 2 non présents dans la liste 1 : ", in_two_not_in_one)

Longueur de la liste 1 :  34853 
Longueur de la liste 2 :  34958 


Élements de la liste 1 non présents dans la liste 2 :  ['16351', '27676', '53239', '53274']  
Élements de la liste 2 non présents dans la liste 1 :  ['2735', '4107', '5051', '5066', '5135', '7018', '8197', '8300', '9033', '9037', '9133', '9143', '9162', '9172', '9237', '10183', '11094', '11135', '11224', '11229', '11230', '11321', '11389', '11406', '15067', '21055', '21168', '21262', '24570', '25130', '25401', '26030', '26123', '26152', '26199', '26227', '26242', '26274', '26374', '27058', '30195', '31081', '31127', '31129', '31146', '31195', '31242', '31549', '31559', '34253', '36095', '38428', '39259', '39324', '39343', '43009', '43252', '43263', '50374', '51470', '52002', '52051', '52157', '52179', '52326', '52373', '52493', '54310', '54457', '54488', '55033', '55039', '55050', '55139', '55162', '55189', '55239', '55307', '55344', '55394', '55506', '57470', '57682', '58237', '62838', '65134', '65180', '65208', '6530

In [6]:
print(len(in_one_not_in_two), len(in_two_not_in_one))

4 109


### III) On définit des fonctions permettant de filtrer le jeu de données, et construire d'autres indicateurs.

Fonction qui sélectionne les données pour un identifiant de ville donné :

In [7]:
def id_select(datas, commune_id, dt1=True):
    if dt1 ==True:
        id_mask = datas["COMMUNE"]==commune_id
    else:
        id_mask = datas["\nCode \nCommune\n"]==commune_id
    return datas[id_mask]

Fonction qui détermine pour une ville (un commune_id donné) les différentes valeurs contenues dans une colonne (un col_name donné) et pour chacune de ces valeurs leur nombre d'occurence. Par exemple pour la ville avec l'id "72181", et la colonne "TRANS", on a 584 fois la valeur 1, 1797 fois la valeur 2, 994 fois la valeur 3, etc...

Également pour la ville d'id "1019" et la colonne "SEXE" on n'a que le label 1 qui est présent trois fois.

In [8]:
def count_(datas, commune_id, col_name, param_filter=False):
    if param_filter==True:
        data_commune = id_select(datas, commune_id) 
    else:
        data_commune = datas
    count = data_commune[col_name].value_counts()
    return count 

In [9]:
count_(raw_datas, "72181", "TRANS", param_filter=True)

5    12333
6     4140
2     1797
3      994
1      584
4      352
Name: TRANS, dtype: int64

In [10]:
count_(raw_datas, "1019", "SEXE") 

1    4086464
2    3846431
Name: SEXE, dtype: int64

On définit une fonction qui pour un jeu de données calcul la somme des poids. (à améliorer)

In [11]:
def weight_sum(datas):
    return datas["IPONDI"].sum()

On décrit dans des listes aux noms explicites les différents labels que l'on est amené à recontrer quand on étudie chacun des items. 

In [12]:
sex_label = [1,2]
iltuu_label = [1, 2, 3, 4, 5, "Z"]
trans_label = [1, 2, 3, 4, 5, 6, "Z"]
voit_label = ['0', '1', '2', '3', "X", "Z"]
na5_label = ['AZ', 'BE', 'FZ', 'GU', "OQ", "ZZ"]

On définit une fonction qui donne le poids de chaque label pour une ville et une colonne données. Par exemple, la ville d'id "1019" dans la catégorie "SEXE" donne un poids total de 15.57 au label 1 (Hommes) et un poids total de zéro au label 2 (Femmes) ce qui s'explique par le fait qu'il n'y a pas d'individus féminins dans le recensement pour cette ville.



De la meme manière pour la ville d'id "72181", dans la catégorie "ILTUU", le poids total accordé aux labels 1 et 2 est de zéro, de 3747.07 pour le label 3, 5947.24 pour le label 4 et 8917.57 pour le label 5.

In [13]:
def count_relative(datas, commune_id, col_name, label, param_filter=False):
    if param_filter == True:
        data_commune = id_select(datas, commune_id)  
    else:
        data_commune = datas
    abs_result = count_(datas, commune_id, col_name)
    missing = label.copy()
    
    sum_weight_dict = {}
    for val in abs_result.index:
        if val in missing:
            val_index = missing.index(val)
            del missing[val_index]  
        mask = data_commune[col_name]==val
        datas = data_commune[mask] 
        sum_weight_dict[str(val)] = weight_sum(datas)
    for m in missing:
        sum_weight_dict[str(m)] = 0
    
    return sum_weight_dict


In [14]:
count_relative(raw_datas, "1019", "SEXE", sex_label, param_filter=True) 

{'1': 15.57692307692307, '2': 0.0}

In [15]:
count_relative(raw_datas, "72181", "ILTUU", sex_label, param_filter=True) 

{'3': 37474.07303480345,
 '4': 5947.24358630968,
 '5': 8917.572202802534,
 '2': 0.0,
 '1': 0.0}

On définit les noms des colonnes de ce qui sera notre nouveau dataframe, avec les indicateurs qui nous intéressent. 

In [16]:
columns = ['COMMUNE', 'MEN_REL', 'WOMEN_REL', 'ILTUU_1_REL', 'ILTUU_2_REL', 'ILTUU_3_REL', 
           'ILTUU_4_REL', 'ILTUU_5_REL', 'ILTUU_Z_REL', 'TRANS_1_REL', 'TRANS_2_REL', 
           'TRANS_3_REL', 'TRANS_4_REL', 'TRANS_5_REL', 'TRANS_6_REL', 'TRANS_Z_REL', 'VOIT_0_REL',
           'VOIT_1_REL', 'VOIT_2_REL', 'VOIT_3_REL', 'VOIT_X_REL', 'VOIT_Z_REL', 'NA5_AZ_REL',
           'NA5_BE_REL', 'NA5_FZ_REL', 'NA5_GU_REL', 'NA5_OQ_REL', 'NA5_ZZ_REL', 'POIDS_VILLE', 'NB_HABITANTS', 'INDICE_DENSITE']

In [46]:
def agregate(datas, datas_2, commune_id):
    Ar = []
    
    bar = progressbar.ProgressBar(maxval=len(commune_id)).start()
    i=0
    
    for c_id in commune_id:
        data_1 = id_select(datas, c_id) 
        data_2 = id_select(datas_2, c_id, dt1=False) 
         
        sex_abs = count_(data_1, c_id, "SEXE")
        iltuu_abs = count_(data_1, c_id, "ILTUU")
        trans_abs = count_(data_1, c_id, "TRANS")
        voit_abs = count_(data_1, c_id, "VOIT")
        na5_abs = count_(data_1, c_id, "NA5")
        
        sex_rel = count_relative(data_1, c_id, "SEXE", sex_label)
        iltuu_rel = count_relative(data_1, c_id, "ILTUU", iltuu_label)
        trans_rel = count_relative(data_1, c_id, "TRANS", trans_label)
        voit_rel = count_relative(data_1, c_id, "VOIT", voit_label)
        na5_rel = count_relative(data_1, c_id, "NA5", na5_label)
         
        MEN_REL, WOMEN_REL = sex_rel["1"], sex_rel["2"] 
        
        POIDS_VILLE = MEN_REL+WOMEN_REL
        
        (ILTUU_1_REL, ILTUU_2_REL, ILTUU_3_REL, ILTUU_4_REL, ILTUU_5_REL, ILTUU_Z_REL) = (iltuu_rel["1"],
        iltuu_rel["2"], iltuu_rel["3"], iltuu_rel["4"], iltuu_rel["5"], iltuu_rel["Z"])
        
        (TRANS_1_REL, TRANS_2_REL, TRANS_3_REL, TRANS_4_REL, TRANS_5_REL, TRANS_6_REL, TRANS_Z_REL) = (trans_rel["1"],
        trans_rel["2"], trans_rel["3"], trans_rel["4"], trans_rel["5"], trans_rel["5"], trans_rel["Z"])
        
        (VOIT_0_REL, VOIT_1_REL, VOIT_2_REL, VOIT_3_REL, VOIT_X_REL, VOIT_Z_REL) = (voit_rel["0"], voit_rel["1"],
        voit_rel["2"], voit_rel["3"], voit_rel["X"], voit_rel["Z"])
        
        (NA5_AZ_REL, NA5_BE_REL, NA5_FZ_REL, NA5_GU_REL, NA5_OQ_REL, NA5_ZZ_REL) = (na5_rel["AZ"],
        na5_rel["BE"], na5_rel["FZ"], na5_rel["GU"], na5_rel["OQ"], na5_rel["ZZ"])
        
         
        try:
            NB_HABITANTS = list(data_2['Population \nmunicipale \n2018'])[0]
        except:
            NB_HABITANTS = 0
        try:
            INDICE_DENSITE = list(data_2['Degré de \nDensité de la commune\n'])[0]
        except:
            INDICE_DENSITE = 0
        
        Ar.append([c_id, MEN_REL, WOMEN_REL, ILTUU_1_REL, ILTUU_2_REL, ILTUU_3_REL, ILTUU_4_REL, 
                  ILTUU_5_REL, ILTUU_Z_REL, TRANS_1_REL, TRANS_2_REL, TRANS_3_REL, TRANS_4_REL, 
                  TRANS_5_REL, TRANS_6_REL, TRANS_Z_REL, VOIT_0_REL, VOIT_1_REL, VOIT_2_REL, VOIT_3_REL, VOIT_X_REL, VOIT_Z_REL, 
                  NA5_AZ_REL, NA5_BE_REL, NA5_FZ_REL, NA5_GU_REL, NA5_OQ_REL, NA5_ZZ_REL, POIDS_VILLE, NB_HABITANTS, INDICE_DENSITE])
        
        bar.update(i)
        i+=1
        
    return np.array(Ar)

In [47]:
filter_datas = agregate(raw_datas, raw_datas_2, communes_ids)

In [48]:
df = pd.DataFrame(filter_datas, columns = columns)
df.head()

,COMMUNE,MEN_REL,WOMEN_REL,ILTUU_1_REL,ILTUU_2_REL,ILTUU_3_REL,ILTUU_4_REL,ILTUU_5_REL,ILTUU_Z_REL,TRANS_1_REL,...,VOIT_Z_REL,NA5_AZ_REL,NA5_BE_REL,NA5_FZ_REL,NA5_GU_REL,NA5_OQ_REL,NA5_ZZ_REL,POIDS_VILLE,NB_HABITANTS,INDICE_DENSITE
0,40018,132.890625,88.59375,49.21875,172.265625,0,0,0,0,14.765625,...,0,24.609375,39.375,24.609375,78.75,54.140625,0,221.484375,378,3
1,40019,260.0,225.0,55.0,430.0,0,0,0,0,35.0,...,0,5.0,125.0,20.0,170.0,165.0,0,485.0,1071,3
2,40020,137.53600000000003,122.80000000000001,19.648,240.688,0,0,0,0,14.736,...,0,9.824,44.208,9.824,88.41600000000001,108.06400000000004,0,260.336,614,3
3,40021,229.11504424778784,190.9292035398232,119.33075221238948,300.7134955752213,0,0,0,0,33.412610619469056,...,0,9.54646017699116,52.50553097345138,42.95907079646022,200.47566371681435,114.5575221238939,0,420.044247787611,863,3
4,40022,86.28787878787885,71.0606060606061,25.378787878787897,131.96969696969708,0,0,0,0,15.227272727272739,...,0,25.378787878787897,20.30303030303032,10.15151515151516,60.90909090909095,40.60606060606064,0,157.34848484848496,402,3


In [49]:
try:
    df = df.astype({"INDICE_DENSITE": int})
except:
    pass
mask_3_4 = df["INDICE_DENSITE"]>2
df_low_densite = df[mask_3_4]

In [50]:
df_low_densite.head()

,COMMUNE,MEN_REL,WOMEN_REL,ILTUU_1_REL,ILTUU_2_REL,ILTUU_3_REL,ILTUU_4_REL,ILTUU_5_REL,ILTUU_Z_REL,TRANS_1_REL,...,VOIT_Z_REL,NA5_AZ_REL,NA5_BE_REL,NA5_FZ_REL,NA5_GU_REL,NA5_OQ_REL,NA5_ZZ_REL,POIDS_VILLE,NB_HABITANTS,INDICE_DENSITE
0,40018,132.890625,88.59375,49.21875,172.265625,0,0,0,0,14.765625,...,0,24.609375,39.375,24.609375,78.75,54.140625,0,221.484375,378,3
1,40019,260.0,225.0,55.0,430.0,0,0,0,0,35.0,...,0,5.0,125.0,20.0,170.0,165.0,0,485.0,1071,3
2,40020,137.53600000000003,122.80000000000001,19.648,240.688,0,0,0,0,14.736,...,0,9.824,44.208,9.824,88.41600000000001,108.06400000000004,0,260.336,614,3
3,40021,229.11504424778784,190.9292035398232,119.33075221238948,300.7134955752213,0,0,0,0,33.412610619469056,...,0,9.54646017699116,52.50553097345138,42.95907079646022,200.47566371681435,114.5575221238939,0,420.044247787611,863,3
4,40022,86.28787878787885,71.0606060606061,25.378787878787897,131.96969696969708,0,0,0,0,15.227272727272739,...,0,25.378787878787897,20.30303030303032,10.15151515151516,60.90909090909095,40.60606060606064,0,157.34848484848496,402,3


In [51]:
df.to_csv('./results/exhaustive/filter_data_reste.csv', sep = ';')
df.to_csv('./results/low_density/filter_data_reste_low_dens.csv', sep = ';')

In [14]:
dt1=pd.read_csv("./results/exhaustive/filter_data_0_5000.csv", sep=";")
dt2=pd.read_csv("./results/exhaustive/filter_data_5001_10000.csv", sep=";")
dt3=pd.read_csv("./results/exhaustive/filter_data_10001_15000.csv", sep=";")
dt4=pd.read_csv("./results/exhaustive/filter_data_reste.csv", sep=";")

dt_f=pd.concat([dt1, dt2, dt3, dt4])
dt_f.head()


,Unnamed: 0,COMMUNE,MEN_REL,WOMEN_REL,ILTUU_1_REL,ILTUU_2_REL,ILTUU_3_REL,ILTUU_4_REL,ILTUU_5_REL,ILTUU_Z_REL,...,VOIT_Z_REL,NA5_AZ_REL,NA5_BE_REL,NA5_FZ_REL,NA5_GU_REL,NA5_OQ_REL,NA5_ZZ_REL,POIDS_VILLE,NB_HABITANTS,INDICE_DENSITE
0,0,1001,201.493986,170.730808,75.923416,296.301378,0.000000,0.000000,0.000000,0,...,0.000000,20.237939,55.318190,20.057390,151.028214,125.583061,0,372.224794,771.0,3.0
1,1,1002,52.057613,57.263374,10.411523,98.909465,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,10.411523,15.617284,46.851852,36.440329,0,109.320988,253.0,4.0
2,2,1004,3229.227582,2738.376419,0.000000,0.000000,2411.084932,41.171982,3515.347088,0,...,249.992447,18.966565,1237.358063,336.628221,2555.745092,1818.906060,0,5967.604002,14204.0,2.0
3,3,1005,434.343434,419.865320,149.607183,704.601571,0.000000,0.000000,0.000000,0,...,0.000000,19.304153,106.172840,120.650954,381.257015,226.823793,0,854.208754,1720.0,3.0
4,4,1006,25.454545,30.545455,0.000000,56.000000,0.000000,0.000000,0.000000,0,...,0.000000,0.000000,10.181818,10.181818,20.363636,15.272727,0,56.000000,112.0,4.0


In [15]:
dt_f = dt_f.drop(dt_f.columns[0], axis=1)  # df.columns is zero-based pd.Index 
dt_f.head() 


,COMMUNE,MEN_REL,WOMEN_REL,ILTUU_1_REL,ILTUU_2_REL,ILTUU_3_REL,ILTUU_4_REL,ILTUU_5_REL,ILTUU_Z_REL,TRANS_1_REL,...,VOIT_Z_REL,NA5_AZ_REL,NA5_BE_REL,NA5_FZ_REL,NA5_GU_REL,NA5_OQ_REL,NA5_ZZ_REL,POIDS_VILLE,NB_HABITANTS,INDICE_DENSITE
0,1001,201.493986,170.730808,75.923416,296.301378,0.000000,0.000000,0.000000,0,30.398846,...,0.000000,20.237939,55.318190,20.057390,151.028214,125.583061,0,372.224794,771.0,3.0
1,1002,52.057613,57.263374,10.411523,98.909465,0.000000,0.000000,0.000000,0,0.000000,...,0.000000,0.000000,10.411523,15.617284,46.851852,36.440329,0,109.320988,253.0,4.0
2,1004,3229.227582,2738.376419,0.000000,0.000000,2411.084932,41.171982,3515.347088,0,238.588621,...,249.992447,18.966565,1237.358063,336.628221,2555.745092,1818.906060,0,5967.604002,14204.0,2.0
3,1005,434.343434,419.865320,149.607183,704.601571,0.000000,0.000000,0.000000,0,53.086420,...,0.000000,19.304153,106.172840,120.650954,381.257015,226.823793,0,854.208754,1720.0,3.0
4,1006,25.454545,30.545455,0.000000,56.000000,0.000000,0.000000,0.000000,0,0.000000,...,0.000000,0.000000,10.181818,10.181818,20.363636,15.272727,0,56.000000,112.0,4.0


In [16]:
dt_f.to_csv('./results/exhaustive/filter_data_final.csv', sep = ';')

In [17]:
dt_f=dt_f[dt_f["INDICE_DENSITE"]>2]
dt_f.shape

(30665, 31)

In [18]:
dt_f.to_csv('./results/low_density/filter_data_low_density_final.csv', sep = ';')